In [1]:
import sys
import os
import pandas as pd
import numpy as np
from datetime import date
import talib
from sklearn.linear_model import *
from sktime.forecasting.base import ForecastingHorizon
from sktime.utils.plotting import plot_series
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error, mean_squared_error
from sklearn.metrics import accuracy_score
from sktime.forecasting.model_selection import SlidingWindowSplitter
from joblib import Parallel, delayed
from itertools import islice
import json
import warnings
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from PIL import Image

/projects/genomic-ml/da2343/cs685/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/projects/genomic-ml/da2343/cs685/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Set a random seed for reproducibility
seed = 42
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    
# Define data transformations
transform = transforms.Compose( 
    [transforms.Resize((64, 64)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 


device = "cpu"

# Save the model
PATH = './model.pth'

# Define the model
class CNNet(nn.Module):
    def __init__(self):
        super(CNNet, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 16, 5) # Input channels, output channels, kernel size
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.conv3 = nn.Conv2d(32, 32, 5)
        self.pool = nn.MaxPool2d(2, 2) # Kernel size, stride
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(86528, 128)
        self.fc2 = nn.Linear(128, 1) 
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Apply the convolutional layers with pooling and dropout
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# Load the trained model
cnn = CNNet()
cnn.load_state_dict(torch.load(PATH))
cnn.to(device)
cnn.eval()

CNNet(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=86528, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [5]:

dataset_name = "EURUSD_H1"
root_data_dir = "/projects/genomic-ml/da2343/ml_project_2/data/EURUSD" 
dataset_path = f"{root_data_dir}/EURUSD_H1_200702210000_202304242100_Update.csv"

# dataset_name = "USDJPY_H1"
# root_data_dir = "/projects/genomic-ml/da2343/ml_project_2/data/USDJPY" 
# dataset_path = f"{root_data_dir}/USDJPY_H1_200705290000_202307282300_Update.csv"

# Load the config file
config_path = "/projects/genomic-ml/da2343/ml_project_2/settings/config.json"
with open(config_path) as f:
  config = json.load(f)
  
# Get the take_profit and stop_loss levels from the config file
config_settings = config["trading_settings"][dataset_name]
tp = config_settings["take_profit"]
sl = config_settings["stop_loss"]
delta = config_settings["delta"]
window_size = config_settings["window_size"]

df = pd.read_csv(dataset_path, index_col=0)
df['Index'] = df.index

y = df[['Close']]
offset = y.index[0]

trades = []

In [7]:
df

,Open,High,Low,Close,Volume,Date_Time,SMA_20,SMA_30,SMA_50,SMA_100,...,STOCH_D,WILLR,BBANDS_Upper,BBANDS_Middle,BBANDS_Lower,AD,ADOSC,VOLUME_RSI,MFI,Index
199,121.190,121.190,120.920,121.000,599,2007.06.08 08:00:00,121.17100,121.182000,121.15340,121.40310,...,60.579423,-71.428571,121.394639,121.1940,120.993361,6.684627e+02,-93.386568,58.552162,46.566499,199
200,120.990,121.120,120.760,120.910,736,2007.06.08 09:00:00,121.14550,121.179667,121.14540,121.39240,...,44.155062,-74.576271,121.420684,121.1280,120.835316,5.457961e+02,-157.838423,61.968270,45.383385,200
201,120.920,121.180,120.760,121.140,639,2007.06.08 10:00:00,121.13450,121.181333,121.13900,121.38390,...,35.553829,-35.593220,121.353298,121.1000,120.846702,1.063082e+03,-5.265235,58.304187,49.520056,201
202,121.130,121.430,121.110,121.290,684,2007.06.08 11:00:00,121.12850,121.186667,121.13900,121.37680,...,41.098357,-20.895522,121.381467,121.1080,120.834533,1.148582e+03,84.834273,59.500570,58.539419,202
203,121.300,121.730,121.280,121.600,813,2007.06.08 12:00:00,121.13650,121.202000,121.14480,121.37330,...,57.839378,-13.402062,121.673864,121.1880,120.702136,1.491848e+03,223.202167,62.796121,67.308413,203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100167,140.718,140.904,140.671,140.860,4789,2023.07.28 19:00:00,139.60255,139.812833,139.94900,140.51665,...,91.293543,-7.346128,141.121547,140.4788,139.836053,9.193598e+06,11498.209386,44.881664,69.698890,100167
100168,140.858,140.918,140.753,140.883,3568,2023.07.28 20:00:00,139.67555,139.838067,139.95838,140.51394,...,93.852682,-1.224633,141.083663,140.6666,140.249537,9.195652e+06,11294.318869,43.944416,67.030259,100168
100169,140.883,141.022,140.866,140.994,3634,2023.07.28 21:00:00,139.76615,139.847300,139.96594,140.51236,...,95.439857,-1.133144,141.220900,140.7562,140.291500,9.197982e+06,10925.353318,44.012475,78.068047,100169
100170,140.993,141.150,140.991,141.088,5100,2023.07.28 22:00:00,139.87715,139.858400,139.97732,140.51132,...,96.363070,-2.695652,141.159715,140.9086,140.657485,9.199104e+06,10138.403044,45.592645,76.692331,100170


In [ ]:

def save_setup_graph(subset_df, position, index):
    green_df = subset_df[subset_df['Close'] > subset_df['Open']].copy()
    green_df["Height"] = green_df["Close"] - green_df["Open"]
    red_df = subset_df[subset_df['Close'] < subset_df['Open']].copy()
    red_df["Height"] = red_df["Open"] - red_df["Close"]
    
    fig = plt.figure(figsize=(8,3))     
    
    ##Grey Lines
    plt.vlines(x=green_df["Index"], 
            ymin=green_df["Low"], 
            ymax=green_df["High"],
            color="green")
    plt.vlines(x=red_df["Index"], 
            ymin=red_df["Low"], 
            ymax=red_df["High"],
            color="orangered")
    ##Green Candles
    plt.bar(x=green_df["Index"], 
            height=green_df["Height"], 
            bottom=green_df["Open"], 
            color="green")
    ##Red Candles
    plt.bar(x=red_df["Index"], 
            height=red_df["Height"], 
            bottom=red_df["Close"], 
            color="orangered")
    
    plt.plot(subset_df["SMA_20"], label="SMA_20")
    plt.plot(subset_df["SMA_30"], label="SMA_30")
    
    close_price = subset_df["Close"].iloc[-1]
    
    tp_eps = tp + delta
    sl_eps = sl + delta
    
    sl_eps = sl
    tp_eps = tp
    
    if position == 1:
        plt.axhspan(close_price, close_price + tp_eps, facecolor="green", xmin= 0.96, alpha=0.9) 
        plt.axhspan(close_price - sl_eps, close_price, facecolor="orangered", xmin= 0.96, alpha=0.9)
    else:
        plt.axhspan(close_price, close_price + sl_eps, facecolor="orangered", xmin= 0.96, alpha=0.9) 
        plt.axhspan(close_price - tp_eps, close_price, facecolor="green", xmin= 0.96, alpha=0.9)
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    
    # name should be the index of the first row in the subset_df
    plt.savefig(f"/projects/genomic-ml/da2343/ml_project_2/cnn/plot.png", dpi=128, bbox_inches="tight")
    # close the figure
    plt.close()
    

def create_trade_order(row, position, tp, sl):
    ask_price = row["Close"]
    tp_price = ask_price + tp if position == 1 else ask_price - tp
    sl_price = ask_price - sl if position == 1 else ask_price + sl

    trade_order = {
        "index": row.name,
        "ask_price": ask_price,
        "take_profit_price": tp_price,
        "stop_loss_price": sl_price,
        "position": position,
        # f"SMA_{timeperiod}": row[f"SMA_{timeperiod}"],
        "MACD": row["MACD"],
        "MACD_Signal": row["MACD_Signal"],
        "MACD_Hist": row["MACD_Hist"],
        "MACD_Crossover_Change" : row["MACD_Crossover_Change"],
        "RSI": row["RSI"],
        "ATR": row["ATR"],
        "ADX": row["ADX"],
        "AROON_Oscillator": row["AROON_Oscillator"],
        "WILLR": row["WILLR"],
        "OBV": row["OBV"],
        "CCI": row["CCI"],
        "PSAR": row["PSAR"],
        "AD": row["AD"],
        "ADOSC": row["ADOSC"],
        "VOLUME_RSI": row["VOLUME_RSI"],
        "MFI": row["MFI"],
        "Date_Time": row["Date_Time"],
        "close_time": None,
        "label": None,
    }
    return trade_order

try:
    # loop through all rows in the dataframe
    for index, row in df.iterrows():
        if index < window_size:
            continue
        
        i = index + offset

        if len(trades) != 0:
            prev_trade = trades[-1]
            # check if the previous trade was a long trade
            if prev_trade["position"] == 1:
                if row["Close"] >= prev_trade["take_profit_price"] and prev_trade["label"] == None:
                    prev_trade["label"] = 1
                    prev_trade["close_time"] = row["Date_Time"]
                    continue
                elif row["Close"] <= prev_trade["stop_loss_price"] and prev_trade["label"] == None:
                    prev_trade["label"] = 0
                    prev_trade["close_time"] = row["Date_Time"]
                    continue
            else:
                if row["Close"] <= prev_trade["take_profit_price"] and prev_trade["label"] == None:
                    prev_trade["label"] = 1
                    prev_trade["close_time"] = row["Date_Time"]
                    continue
                elif row["Close"] >= prev_trade["stop_loss_price"] and prev_trade["label"] == None:
                    prev_trade["label"] = 0
                    prev_trade["close_time"] = row["Date_Time"]
                    continue
                    
            if prev_trade["label"] == None:
                continue
    
        macd_crossover_change = row["MACD_Crossover_Change"]
        if macd_crossover_change > 0 or macd_crossover_change < 0:
            current_position = 1 if macd_crossover_change > 0 else 0
            # TODO: Dummy
            # local_order = create_trade_order(row, current_position, tp, sl)
            # trades.append(local_order) 

            # TODO: ML
            subset_df = df.loc[i-window_size:i]
            save_setup_graph(subset_df, current_position, i)
            # use the model to predict 1 or 0
            image = Image.open("plot.png").convert("RGB")
            image = transform(image)
            image = image.unsqueeze(0)
            image = image.to(device)
            # Get the model output
            output = cnn(image)
            output_item = output.item()
            threshold = 0.5
            pred = 1 if output_item > threshold else 0
            # use that to execute a trade order
            if pred == 1:
                print("output_item: ", output_item)
                local_order = create_trade_order(row, current_position, tp, sl)
                trades.append(local_order) 
except Exception as e:
    print(e)
    
trades_df = pd.DataFrame(trades)
# save the trades dataframe to a csv file
trades_df.to_csv(f"ml_2_trades_threshold_0.5_seq_{dataset_name}_2007_2023.csv", index=False)
# trades_df.to_csv(f"dummy_trades_seq_{dataset_name}_2007_2023.csv", index=False)
print("Done!")